In [ ]:
# automatically reload packages every time we run import, so that changes are included
if __name__ == "__main__":
    %load_ext autoreload
    %autoreload 2

# 2017 GVA Publication

#### Import packages and define paths to directories

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import platform

# add root directory to sys.path so we that our packages can be found
for path in ['../..', '']:
    if path not in sys.path:
        sys.path.append(path)

# import package functions
from gva_data_processing import *

if platform.system() == 'Darwin':
    shared_drive = '/Volumes/Data/EAU/Statistics/'
elif platform.system() == 'Windows':
    shared_drive = 'Z:/'

# set path to raw data excel file
path = shared_drive + 'Economic Estimates/2018 publications/GVA/CP Working tables/DP_Working_file_dcms_V11 2018 Data.xlsx'

## Part 1 - Read in, clean, and aggregate data
This section makes use of the source code in the GVA package's src folder

#### Read in and clean up raw data in excel file

In [ ]:
abs = read_abs(path, cols=list(range(13, 24)))
charities = read_charities(path, rows=list(range(0,8)))
tourism = read_tourism(path)
gva = read_gva(path, rows=list(range(9,37)))
sic91 = read_sic91(path)

#### Combine sic level data read in above into a single dataset

In [ ]:
combined_gva = combine_gva(abs, gva, sic91)

#### Aggregate data to sector level
we want the data all in a single dataset so that sector totals can be easily added to subsector breakdowns, and we do not have to store the values twice, which could be confusing.

In [ ]:
agg = aggregate_data(combined_gva, gva, tourism, charities)

#### Read and append CVM data 


In [ ]:
cvm = pd.read_excel(path, sheet_name = 'CVM')
cvm = cvm.stack().reset_index()
cvm.columns = ['year', 'sector', 'cvm']
cvm['sub-sector'] = 'All'

In [ ]:
agg2 = pd.concat([agg, cvm], sort=True)
agg2
agg2.columns = ['cvm', 'cp', 'sector', 'sub-sector', 'year']
cols=['year', 'sector', 'sub-sector', 'cp', 'cvm']
agg2 = agg2[cols]

#### Save aggregated data to ouputs directory

In [ ]:
agg.to_csv('gva_aggregate_data_2017.csv', index=False)
#agg2.to_csv('gva_aggregate_data_2017.csv', index=False)

# Note that there are duplicates in the data for example, you can see C.I and publishing more than once in the data but this will be taken care of at a later stage

## Part 2 - Define summary tables and variables
This section makes used of the report_maker package

#### Read in aggregate data

This demonstrates that, once the CSV has been generated and published, all the the publication outputs can be created from it, using the below code.

In [ ]:
agg = pd.read_csv('gva_aggregate_data_2017.csv')

In [ ]:
temp = agg.set_index(['year', 'sector', 'sub-sector'])
current_year = 2018
ci_current = temp.loc[[current_year, 'Creative Industries', 'Crafts']]['cvm']

In [ ]:
agg.set_index(['year', 'sector', 'sub-sector'])

#### Create some summary tables
the `make_table()` function simply make time series for different subsets of the data

#### Create dictionary to be populate html template

In [ ]:
context = {}

In [ ]:
import json
def chart_json(tb):
    # gva_current_extended = gva_current_extended.reset_index()
    # convert column names to strings to ensure order is maintained
    #gva_current_extended.columns = [str(i) for i in list(gva_current_extended.columns)]
    tb = tb.transpose()
    tb = tb.reset_index()
    tb = tb.to_json(orient='table', index=False)
    myd = json.loads(tb)
    tb_json = json.dumps(myd['data'])
    return tb_json

#### Define tables

In [ ]:
gva_current = make_table(agg, 'All')
gva_current_indexed = make_table(agg, 'All', indexed=True)
creative = make_table(agg, 'Creative Industries')
digital = make_table(agg, 'Digital Sector')
culture = make_table(agg, 'Cultural Sector')
gva_current.loc['All DCMS sectors', 2017]

In [ ]:
#make_table(agg, 'All', indexed=True).loc[['All DCMS sectors', 'UK'],:]

In [ ]:
#make_table(agg, 'All', indexed=True)

In [ ]:
tb = make_table(agg, 'All')
tb.loc['All DCMS sectors (excl Tourism)'] = tb.loc['All DCMS sectors'] - tb.loc['Tourism']
tb = tb.loc[['All DCMS sectors (excl Tourism)', 'UK'],:]
data = tb.copy()
tb.loc[:, 2010] = 100
for y in range(2011, 2018):
    tb.loc[:, y] = data.loc[:, y] / data.loc[:, 2010] * 100
tb = round(tb, 1)
fig_2_1 = chart_json(tb)

In [ ]:


fig_3_1 = make_table(agg, 'All', indexed=True).loc[['Civil Society (Non-market charities)', 'UK'],:]
fig_3_1 = chart_json(fig_3_1)

fig_3_2 = make_table(agg, 'All', indexed=True).loc[['Creative Industries', 'UK'],:]
fig_3_2 = chart_json(fig_3_2)

fig_3_3 = make_table(agg, 'All', indexed=True).loc[['Cultural Sector', 'UK'],:]
fig_3_3 = chart_json(fig_3_3)

fig_3_4 = make_table(agg, 'All', indexed=True).loc[['Digital Sector', 'UK'],:]
fig_3_4 = chart_json(fig_3_4)

fig_3_5 = make_table(agg, 'All', indexed=True).loc[['Gambling', 'UK'],:]
fig_3_5 = chart_json(fig_3_5)

fig_3_6 = make_table(agg, 'All', indexed=True).loc[['Sport', 'UK'],:]
fig_3_6 = chart_json(fig_3_6)

fig_3_7 = make_table(agg, 'All', indexed=True).loc[['Telecoms', 'UK'],:]
fig_3_7 = chart_json(fig_3_7)

In [ ]:
tb = make_table(agg, 'All', indexed=True).loc[['Tourism', 'UK'],:]
#tb.columns = [2010, 2011, 2012, 2013, 2014, '2015 (1)', '2016 (1)', '2017 (1)']
#tb.columns.name = 'year'
fig_3_8 = chart_json(tb)

#### Define individual stats

In [ ]:
#gva_current.to_excel('gva_current_nb.xlsx')

#This step saves the table we just created. Use this to QA the output against the manual working tables. Note that you may have to change the code in functions.py to QA against the unrounded data

In [ ]:
perc_change_2010 = (gva_current.loc[:,2017] / gva_current.loc[:,2010] - 1) * 100
perc_change_last_year = (gva_current.loc[:,2017] / gva_current.loc[:,2016] - 1) * 100
perc_of_uk = (gva_current.loc[:, 2017] / gva_current.loc['UK', 2017]) * 100
uk_current_total = gva_current.loc['UK', 2017]

#### Extended tables

In [ ]:
tb = round(make_table(agg, 'All'), 100)
# add extra all dcms row
tb.loc['All DCMS sectors (excl Tourism)'] = tb.loc['All DCMS sectors'] - tb.loc['Tourism']
tb = round(tb, 1)
# reorder so extra row is in correct place
tb = tb.loc[['Civil Society (Non-market charities)', 'Creative Industries',
       'Cultural Sector', 'Digital Sector', 'Gambling', 'Sport', 'Telecoms',
       'Tourism', 'All DCMS sectors (excl Tourism)', 'All DCMS sectors', 'UK']]

# add percentage columns
perc_change_2010 = (tb.loc[:,2017] / tb.loc[:,2010] - 1) * 100
perc_change_last_year = (tb.loc[:,2017] / tb.loc[:,2016] - 1) * 100
perc_of_uk = (tb.loc[:, 2017] / tb.loc['UK', 2017]) * 100
uk_current_total = tb.loc['UK', 2017]

tb['% change 2016-2017'] = round(perc_change_last_year, 1)
tb['% change 2010-2017'] = round(perc_change_2010, 1)
tb['% of UK GVA 2016'] = round(perc_of_uk, 1)
myindex = tb.index.tolist()
myindex[0] = '<p>' + myindex[0] + '<sup>2</sup></p>'
myindex[7] = '<p>' + myindex[7] + '<sup>7</sup></p>'
myindex[9] = '<p>' + myindex[9] + '<sup>7</sup></p>'
tb.index = myindex
tb.index.name = 'Sector'
tb = tb.reset_index()
mycols = tb.columns.tolist()
mycols[1] = '<p>' + str(mycols[1]) + '<sup>(r)</sup></p>'
mycols[2] = '<p>' + str(mycols[2]) + '<sup>(r)</sup></p>'
mycols[3] = '<p>' + str(mycols[3]) + '<sup>(r)</sup></p>'
mycols[4] = '<p>' + str(mycols[4]) + '<sup>(r)</sup></p>'
mycols[5] = '<p>' + str(mycols[5]) + '<sup>(r)</sup></p>'
mycols[6] = '<p>' + str(mycols[6]) + '<sup>(r)</sup></p>'
mycols[7] = '<p>' + str(mycols[7]) + '<sup>(r)</sup></p>'
mycols[8] = '<p>' + str(mycols[8]) + '<sup>(p)1</sup></p>'

tb.columns = mycols
# convert column names to strings to ensure order is maintained
#gva_current_extended.columns = [str(i) for i in list(gva_current_extended.columns)]
tb_2_1 = tb.to_json(orient='split', index=False)

In [ ]:
tb = round(make_table(agg, 'All'), 1)
tb.loc['All DCMS sectors (excl Tourism)'] = tb.loc['All DCMS sectors'] - tb.loc['Tourism']
# reorder so extra row is in correct place
tb = tb.loc[['Civil Society (Non-market charities)', 'Creative Industries',
       'Cultural Sector', 'Digital Sector', 'Gambling', 'Sport', 'Telecoms',
       'Tourism', 'All DCMS sectors (excl Tourism)', 'All DCMS sectors', 'UK']]
#tb.loc[['All DCMS sectors (exc. Tourism)', 'UK'],:]
#fig_2_1 = chart_json(fig_2_1)

In [ ]:
gva_current_json = round(make_table(agg, 'All'), 1)
gva_current_json = chart_json(gva_current_json)

#### Overlap table

In [ ]:
overlap_tb = pd.read_excel(path, sheet_name = 'Overlaptable')
#overlap_tb = round(overlap_tb, 1) # waiting for access to shared drive
overlap_tb = overlap_tb.to_json(orient='split', index=False)

#### Annex table

In [ ]:
annex_b = pd.read_excel(path, sheet_name = 'AnnexB')
annex_b = annex_b.to_json(orient='split', index=False)

#### Table 4.1

In [ ]:
tb = pd.read_excel(path, sheet_name = 'CVM_table')
tb = round(tb, 1)

tb = tb.set_index('Sector')
copy_4_1 = tb.copy()

myindex = tb.index.tolist()
myindex[6] = '<p>' + myindex[6] + '<sup>7</sup></p>'
myindex[8] = '<p>' + myindex[8] + '<sup>2</sup></p>'
tb.index = myindex
tb.index.name = 'Sector'
tb = tb.reset_index()
mycols = tb.columns.tolist()
mycols[1] = '<p>' + str(mycols[1]) + '<sup>(r)</sup></p>'
mycols[2] = '<p>' + str(mycols[2]) + '<sup>(r)</sup></p>'
mycols[3] = '<p>' + str(mycols[3]) + '<sup>(r)</sup></p>'
mycols[4] = '<p>' + str(mycols[4]) + '<sup>(r)</sup></p>'
mycols[5] = '<p>' + str(mycols[5]) + '<sup>(r)</sup></p>'
mycols[6] = '<p>' + str(mycols[6]) + '<sup>(r)</sup></p>'
mycols[7] = '<p>' + str(mycols[7]) + '<sup>(r)</sup></p>'
mycols[8] = '<p>' + str(mycols[8]) + '<sup>(p)1</sup></p>'

tb.columns = mycols

tb_4_1 = tb.to_json(orient='split', index=False)

In [ ]:
copy_4_1.loc[['All DCMS sectors (excl Tourism)', 'UK'],[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]]

In [ ]:
tb = copy_4_1.loc[['All DCMS sectors (excl Tourism)', 'UK'],[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]]
tb.columns.name = 'year'
tb.loc[:, 2010] = 100
for y in range(2011, 2018):
    tb.loc[:, y] = data.loc[:, y] / data.loc[:, 2010] * 100
tb = round(tb, 5)
fig_4_1 = chart_json(tb)

In [ ]:
tb = gva_current_indexed
tb = tb.drop(['All DCMS sectors', 'UK'])
fig_4_2 = chart_json(tb)

#### Convert data for charts

## Part 3 - Produce written reports
This section makes used of the report_maker package

### Build Written Report

read json template in as python dict - update according, then convert back to json.

In [ ]:
# considering just passing the global environment to build so we don't have to specify this, or do all of the
# above within a new environment to convert to dict. use context.append().
context = {
    # publication info
    'release_date': '29 November 2017',
    
    
    # infographics
    'money_bag': {'text': '£268'},
    'donut': {'text': '14.6'},
    'up_arrow_1': {'text': '3.4%'},
    
    # json data
    'fig_2_1': fig_2_1,
    'tb_2_1': tb_2_1,
    'tb_2_2': overlap_tb,
    
    'fig_3_1': fig_3_1,
    'fig_3_2': fig_3_2,
    'fig_3_3': fig_3_3,
    'fig_3_4': fig_3_4,
    'fig_3_5': fig_3_5,
    'fig_3_6': fig_3_6,
    'fig_3_7': fig_3_7,
    'fig_3_8': fig_3_8,
    
    'fig_4_1': fig_4_1, # needs updating
    'tb_4_1': tb_4_1,
    'fig_4_2': fig_4_2,
    'annex_b': annex_b,
        
}

In [ ]:
from report_maker import build
build(context)

## Part 3 - Create Excel Tables
This section makes use of the spreadsheet_maker package. By default it will look for templates in publication_dir/spreadsheets/templates
https://github.com/pytest-dev/pytest/issues/2268

`make_template()` saves a little mannual work, and helps make excel files more predictable for programatically accessing sheets, and more consistent across publications. Rerunning with the same filenames will overwritting not overwrite by default.

After generating templates, make any mannual adjustments and save. Try to keep these simple and avoid more complex things like images, cell merging, complicated formatting etc, since openpyxl can only read and write the basic feature of an excel file.
`populate_template()` populate sheets according the a dictionary of sheets and associated dataframes, passed to tables.

In [ ]:
from spreadsheet_maker import make_template, populate_template

# sector tables
fn = 'GVA_sector_tables.xlsx'
tables={
    "1.1 - GVA current (£bn)": gva_current,
    "1.1a - GVA current (2010=100)": gva_current_indexed,
    "2.1 - GVA CVM (£bn)": None,
    "2.1a - GVA CVM (2010=100)": None,
}
# make list of sheet names from the keys from tables
make_template(fn=fn, sheets=list(tables.keys()), overwrite=True)
populate_template(fn=fn, tables=tables)

# subsector tables
fn = 'GVA_subsector_tables.xlsx'
tables={
    "1 - Creative Industries-current": creative,
    "2 - Digital Sector-current": digital,
    "3 - Cultural Sector-current": culture,
    "4 - Computer Games-current": None,
    "5 - Creative Industries-CVM": None,
    "6 - Digital Sector-CVM": None,
    "7 - Cultural Sector-CVM": None,
}
make_template(fn=fn, sheets=list(tables.keys()), overwrite=True)
populate_template(fn=fn, tables=tables)


## Part 4 - Testing

#### Dictionary of summary tables for use by the test script

In [ ]:
summary_tables = {
    'gva_current': make_table(agg, 'All'),
    'gva_current_indexed': make_table(agg, 'All', indexed=True),
    'creative': make_table(agg, 'Creative Industries'),
    'digital': make_table(agg, 'Digital Sector'),
    'culture': make_table(agg, 'Cultural Sector'),
}


In [ ]:
summary_tables['digital']

In [ ]:
gva_current_indexed.to_excel('gva_current_indexed_nb.xlsx')
creative.to_excel('creative_nb.xlsx')
digital.to_excel('digital_nb.xlsx')
culture.to_excel('culture_nb.xlsx')